# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [1]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display


In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [3]:
# set up environment
load_dotenv(override=True)
openai = OpenAI()



In [4]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [5]:
system_prompt= """
You are a highly skilled programming assistant. 
Your task is to explain any code provided by the user clearly, accurately, and concisely. 
Focus on purpose, logic, and execution flow. Avoid unnecessary commentary and only describe what the code does and how it works.
Response in Markdown.
"""

In [6]:
# Get gpt-4o-mini to answer, with streaming
stream = openai.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": question}
        ],
    stream=True
)
response = ""
display_value = display(Markdown(""), display_id=True)

for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_value.display_id)
    

This code is a generator expression that yields values from a set created based on the `books` collection. Here's a breakdown of its purpose and execution flow:

1. **Set Comprehension**:
   - The `{book.get("author") for book in books if book.get("author")}` part creates a set of authors. It iterates over each `book` in the `books` iterable.

2. **Filtering**:
   - The condition `if book.get("author")` ensures that only books with a non-`None` or non-empty author value are included in the set. This means if a book does not have an author specified, it will be skipped.

3. **Yielding Values**:
   - The `yield from` statement is used to yield each value from the resulting set of authors. This allows the surrounding function to act as a generator, returning each author one by one.

4. **Purpose**:
   - The overall purpose of this code is to provide a generator that produces a unique collection of book authors, efficiently skipping any entries that lack an author.

In summary, this code is designed to produce a unique list of authors from a collection of book entries, omitting any entries that do not have an associated author.

In [19]:
# Get Llama 3.2 to answer
from ollama import Client

api_key = os.environ.get('OLLAMA_API_KEY')
OLLAMA_BASE_URL = "https://www.ollama.com"

ollama = Client(
    host=OLLAMA_BASE_URL,
    headers={'Authorization': 'Bearer ' + api_key}
)

stream = ollama.chat(
    model='gpt-oss:120b',
    messages=[
        {"role":"system", "content": system_prompt},
        {"role":"user", "content": question}
        ],
    stream=True
)

response = ""
display_value = display(Markdown(""), display_id=True)

for chunk in stream:
    response += chunk['message']['content']
    update_display(Markdown(response), display_id=display_value.display_id)

# for chunk in stream:
#     response += chunk.choices[0].delta.content or ''
#     update_display(Markdown(response), display_id=display_value.display_id)

## What the statement does  

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. **Creates a set**  
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```  
   * Iterates over the iterable `books`.  
   * For each `book` dictionary it calls `book.get("author")`.  
   * The `if book.get("author")` guard filters out entries where the `"author"` key is missing or its value is falsy (`None`, `''`, `0`, etc.).  
   * The result of `book.get("author")` is added to a **set**, which automatically removes duplicates.  
   After the comprehension finishes you have a set of **unique, non‑empty author names**.

2. **Yields each element of that set**  
   `yield from <iterable>` is a shortcut inside a generator function that forwards (or “delegates”) all values produced by the given iterable to the caller of the generator.  
   In this case it iterates over the set produced in step 1 and yields each author — one at a time.

So, when this line appears inside a generator function, the function will output each distinct author found in `books`, skipping books that lack an author entry.

---

## Why it is written this way  

| Reason | Explanation |
|--------|-------------|
| **`yield from`** | Removes the need for an explicit loop (`for a in authors: yield a`). It makes the generator concise and clearly expresses “forward everything from this iterable”. |
| **Set comprehension** (`{ … }`) | Guarantees that each author is yielded only once, because sets cannot contain duplicates. |
| **`book.get("author")` with a guard** | Safely accesses the `"author"` key (no `KeyError`) and filters out falsy values, ensuring the set contains only meaningful author strings. |
| **Combined in one line** | Provides a compact, readable one‑liner that both extracts the unique authors and yields them in a generator context. |

---

## Execution flow (step‑by‑step)

1. The generator function is called; execution stops at the `yield from` line.
2. The set comprehension runs:
   - For each `book` in `books`:
     - `author = book.get("author")`
     - If `author` is truthy, add it to the temporary set.
3. Once the set is built, `yield from` obtains an iterator over that set.
4. The generator yields each `author` value in arbitrary order (the order of a set is not guaranteed).
5. When the set is exhausted, the `yield from` statement finishes and control returns to the caller of the generator.

---

### Quick example

```python
def authors(books):
    yield from {b.get("author") for b in books if b.get("author")}

books = [
    {"title": "A", "author": "Alice"},
    {"title": "B", "author": "Bob"},
    {"title": "C", "author": "Alice"},   # duplicate author
    {"title": "D"}                       # no author key
]

print(list(authors(books)))   # → ['Alice', 'Bob'] (order may vary)
```

The function returns a generator that yields `"Alice"` and `"Bob"` once each, ignoring the duplicate and the book without an author.